In [6]:
import pandas as pd
import numpy as np
from tasks.pipeline import indicator_pipeline
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from index.utils import ISO_to_Everything

In [70]:
def preprocess():
    df = (
        pd.read_csv('data/indicator/ME3/raw/ME3.2_FAO.M.csv')
          .groupby(['Area', 'Year', 'Element'])['Value'].sum().reset_index()
          .pivot(index=['Area', 'Year'], columns='Element', values='Value')
    )
    
    df['Value'] = df['Loss'] / df['Production'] * 100
    return df['Value'].reset_index().rename(columns={'Area': 'Country'})
df = preprocess()

In [71]:
df

,Country,Year,Value
0,Afghanistan,2014,7.173259
1,Afghanistan,2015,6.810589
2,Afghanistan,2016,7.301008
3,Afghanistan,2017,6.726861
4,Afghanistan,2018,6.114663
...,...,...,...
870,Zimbabwe,2014,3.866814
871,Zimbabwe,2015,4.149432
872,Zimbabwe,2016,4.217453
873,Zimbabwe,2017,5.564645


# Food security

In [7]:
indicator_pipeline('GS1')

Downloading..
Saving at data/indicator/GS1/raw/GS1_SDG.json: DONE
DONE
Preprocessing...
PreProcessing GS1 Manual files: Error:  'NoneType' object is not subscriptable
PreProcessing data/indicator/GS1/raw/GS1_SDG.json: Saving at data/indicator/GS1/preprocessed/GS1_SDG.csv
Done
Processing...
Processing GS1: 
	 Processing GS1_SDG.csv
	 Imputation:DONE
	 Outlier removal:DONE
	 Formatting:DONE
	 saving at data/indicator/GS1/processed/GS1_SDG.csv


In [52]:
data = pd.read_csv('data/indicator/GS1/processed/GS1_SDG.csv')
ST = pd.DataFrame({"Indicator": ['GS1'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 0, 'Target 2': np.nan}).set_index('Indicator')
df = (
    data.groupby('Year')
        .apply(lambda x: GreenGrowthScaler().normalize(x.rename(columns={'Value': 'GS1'}).set_index('ISO')[['GS1']], ST)).reset_index()
        .pivot(index=['ISO'], columns=['Year'], values='GS1')
        .dropna(axis=1, how='all')
        .reset_index()
)
#ISO_to_Everything(df)#.to_csv('data/indicator/GS1/processed/food_insecurity_ts.csv')

In [ ]:
# # This part is added to data/indicator/TMP/preprocess.py

# def process_TMP():
#     df = (
#         pd.read_csv('data/indicator/TMP/raw/TMP_IEA.M.csv')
#           .melt(id_vars=['Country', 'Mode/vehicle type', 'Indicator'], var_name=['Year'], value_name='Value')
#           .rename(columns={'Mode/vehicle type': 'mode'})
#           .assign(Indicator=lambda x: x.Indicator.str.strip())
#           .query("mode == 'Total passenger transport' and Indicator == 'Passenger-kilometres energy intensity (MJ/pkm)'")
#           .drop(columns=['mode', 'Indicator'])
#           .dropna()
#     )
#     return df
    
    
# config_TMP = {'Variable': 'TMP',
#              'function': process_TMP,
#              'Description': 'Total passenger transport Passenger-kilometres energy intensity (MJ/pkm)',
#              'Source': 'IEA',
#              'URL': 'https://www.iea.org/data-and-statistics/data-product/energy-efficiency-indicators'}

In [ ]:
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler

indicator = (pd.read_csv('data/indicator/TMP/processed/TMP_origin.M.csv')
               .query("Year == 2019")
               .set_index("ISO")[['Value']]
                .rename(columns={"Value": 'TMP'})
            ) # Data Frame with indicators here there is just "TMP", must be for a single year ! 

ST = pd.DataFrame({"Indicator": ['TMP'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 1.104, 'Target 2': np.nan}).set_index('Indicator')
# ST stands for sustainable target:
# - Number of targets is almost always 1 so leave it as 1 by default
# - Relation is wether high value is good/bad for the environnment. Here, lower energy intensity is better so the relation is negative
# - Target 1 and 2 are for the value of the target. To compute the target take the average of the 5 best countries

In [ ]:
Normalized_Indicator = GreenGrowthScaler().normalize(indicator, ST) # Call this and that's it.
Normalized_Indicator

In [ ]:
Normalized_Indicator.to_csv('data/indicator/TMP/processed/TMP_Normalized_origin.M.csv') # You can share this

In [ ]:
pd.read_csv('data/sustainable_targets/ST_2020.csv', index_col=0) # Just so you can see what it looks like for other indicators

#  Universal acess

In [45]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler

def process():
    df_1 = pd.read_csv('data/indicator/GS3/raw/Rural access index.csv').dropna(axis=1).rename(columns={'Value': 'Rural access index'})
    df_2 = pd.read_csv('data/indicator/GS3/raw/Percentage of female workers in transport.csv').dropna(axis=1).rename(columns={'Value': 'Percentage female workers in transport'})
    df_3 = pd.read_csv('data/indicator/GS3/raw/Rapid Transit to Resident Ratio.csv').dropna(axis=1).rename(columns={'Value': 'Rapid Transit to Resident Ratio'})

    
    df = pd.concat([df_1.set_index(['Country']), df_2.set_index(['Country']), df_3.set_index(['Country'])], axis=1).drop(columns=['Year']).dropna()

    
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)
    return df_norm.mean(axis=1).to_frame(name='Value').assign(Year=2020).reset_index()

In [46]:
df = process()
#df = add_ISO(df.reset_index())
#ISO_to_Everything(df)[['Country', 'Universal Acess']].to_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')

In [47]:
df

,Country,Value,Year
0,Afghanistan,20.387500,2020
1,Algeria,32.266960,2020
2,Angola,19.790086,2020
3,Argentina,32.540616,2020
4,Armenia,54.921645,2020
...,...,...,...
99,United States,68.638566,2020
100,Uruguay,42.818966,2020
101,Vietnam,39.684348,2020
102,Zambia,9.463362,2020


In [ ]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler
from processing.utils import add_ISO
from index.utils import ISO_to_Everything

def process():
    df = pd.read_csv('data/indicator/GS3/raw/Logistics performance index.csv').dropna(axis=1).rename(columns={'Value': 'Logistics performance index'}).set_index('Country')

    
    
    
    ST = pd.DataFrame({"Indicator": ['Logistics performance index'],
                   "Number of targets": [1],
                   "Relation": ['positive'],
                   'Target 1': [4.069],
                   'Target 2': [np.nan]}).set_index('Indicator')

    #df_norm = GreenGrowthScaler().normalize(df[['Logistics performance index']], ST) # Call this and that's it.
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)

    return df_norm

In [ ]:
df = process()
df = add_ISO(df['Logistics performance index'].reset_index())

In [ ]:
ISO_to_Everything(df)[['Country', 'Logistics performance index']].to_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')